# LeNet

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：LeNet [1]。这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。LeNet的网络结构如下图所示。

## LeNet模型

LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现LeNet模型。

In [1]:
import time
import torch
import torch.nn as nn
from torch import optim

import sys
sys.path.append("..") 
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.enabled = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.Sequential_cov = nn.Sequential(
            nn.Conv2d(1, 6, 5) ,# in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2 ,2 ) ,#  # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2 ,2 )) #  # kernel_size, stride
        self.Sequential_linear = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    
    def forward(self, x):
        x = self.Sequential_cov(x)
        return self.Sequential_linear(x.view(x.shape[0], -1))
    

In [2]:
input_test = torch.randn(5,1,28,28)
net = LeNet()
out = net(input_test)
print(out)

tensor([[ 0.4499,  0.0426,  0.4224, -0.1662,  0.3933, -0.0846, -0.0712, -0.3690,
          0.1842,  0.0105],
        [ 0.4502,  0.0422,  0.4224, -0.1661,  0.3931, -0.0846, -0.0711, -0.3688,
          0.1843,  0.0107],
        [ 0.4501,  0.0423,  0.4225, -0.1659,  0.3931, -0.0846, -0.0711, -0.3687,
          0.1842,  0.0107],
        [ 0.4500,  0.0424,  0.4225, -0.1662,  0.3933, -0.0846, -0.0711, -0.3689,
          0.1844,  0.0105],
        [ 0.4500,  0.0425,  0.4227, -0.1657,  0.3932, -0.0848, -0.0711, -0.3688,
          0.1843,  0.0105]], grad_fn=<AddmmBackward>)


In [3]:
print(net)

LeNet(
  (Sequential_cov): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Sequential_linear): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [4]:
batch_size = 256
import dl_utils
train_iter, test_iter = dl_utils.load_data_fashion_mnist(batch_size=batch_size)


因为卷积神经网络计算比多层感知机要复杂，建议使用GPU来加速计算。因此，我们对3.6节（softmax回归的从零开始实现）中描述的`evaluate_accuracy`函数略作修改，使其支持GPU计算。

In [5]:

def evaluate_accuracy(data_iter, net, 
                      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n


def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [6]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.9078, train acc 0.295, test acc 0.579, time 5.7 sec
epoch 2, loss 0.4722, train acc 0.644, test acc 0.694, time 4.7 sec
epoch 3, loss 0.2513, train acc 0.720, test acc 0.737, time 4.4 sec
epoch 4, loss 0.1638, train acc 0.749, test acc 0.753, time 4.4 sec
epoch 5, loss 0.1202, train acc 0.765, test acc 0.762, time 4.3 sec


不用cudnn 
training on  cuda
epoch 1, loss 1.8613, train acc 0.319, test acc 0.580, time 9.6 sec
epoch 2, loss 0.4780, train acc 0.628, test acc 0.687, time 4.1 sec
epoch 3, loss 0.2591, train acc 0.713, test acc 0.729, time 4.0 sec
epoch 4, loss 0.1712, train acc 0.740, test acc 0.746, time 4.0 sec
epoch 5, loss 0.1261, train acc 0.756, test acc 0.759, time 4.0 sec

## 小结

- 卷积神经网络就是含卷积层的网络。
- LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。

